# El problema del viajante

Dada una lista de ciudades y las distancias entre cada par de ciudades determinar la ruta más pequeña posible que visite cada ciudad y regrese a la ciudad de origen.

El problema en esta situación consiste en ser un problema de combinatoria donde el costo computacional de evaluar todas las combinaciones posibles aumenta mientras se incrementa la cantidad de ciudades.

Para obtener lo datos para la resolución y la heurística puede utilizarse la herramienta Google Maps.

# Primera parte

Resolver el problema utilizando un método de fuerza bruta para los siguientes destinos:

- San Rafael
- El sosneado
- Jaime Prats
- Gral. Alvear


- **SR - ES:** 137 km
- **SR - JP:** 70,1 km
- **SR - GA:** 84,8 km
- **ES - JP:** 199 km
- **ES - GA:** 214 km
- **JP - GA:** 15 km

Mostrar el costo de todas las rutas

In [1]:
import itertools

sr_distances = {"ES": 137, "JP": 70.1, "GA": 84.8}
es_distances = {"SR": 137, "JP": 199, "GA": 214}
jp_distances = {"SR": 70.1, "ES": 199, "GA": 15}
ga_distances = {"SR": 84.8, "ES": 214, "JP": 15}


def get_distances_for_city(city: str) -> dict:
    if city == "SR":
        distances = sr_distances
    elif city == "ES":
        distances = es_distances
    elif city == "JP":
        distances = jp_distances
    elif city == "GA":
        distances = ga_distances
    return distances


def calculate_distance_of_route(route):
    # San rafael a la primera ciudad
    distance = sr_distances[route[0]]

    # Primer ciudad a segunda
    distances_for_first_city = get_distances_for_city(route[0])
    distance += distances_for_first_city[route[1]]

    # Segunda ciudad a tercera
    distances_for_second_city = get_distances_for_city(route[1])
    distance += distances_for_second_city[route[2]]

    # Tercer ciudad a Sanra
    distances_for_third_city = get_distances_for_city(route[2])
    distance += distances_for_third_city["SR"]
    return distance


min_distance = float('inf')
shortest_route = None

for route in itertools.permutations(["ES", "JP", "GA"]):
    distance_of_current_route = calculate_distance_of_route(route)
    route = list(route)
    route.insert(0, 'SR')
    route.append('SR')
    print('Ruta: ' + ' -> '.join(route))
    print(f'Distancia: {distance_of_current_route} km\n')
    if distance_of_current_route < min_distance:
        min_distance = distance_of_current_route
        shortest_route = route

print('Ruta mas efectiva: ' + ' -> '.join(shortest_route))
print(f'Distancia estimada: {min_distance} km')

Ruta: SR -> ES -> JP -> GA -> SR
Distancia: 435.8 km

Ruta: SR -> ES -> GA -> JP -> SR
Distancia: 436.1 km

Ruta: SR -> JP -> ES -> GA -> SR
Distancia: 567.9 km

Ruta: SR -> JP -> GA -> ES -> SR
Distancia: 436.1 km

Ruta: SR -> GA -> ES -> JP -> SR
Distancia: 567.9 km

Ruta: SR -> GA -> JP -> ES -> SR
Distancia: 435.8 km

Ruta mas efectiva: SR -> ES -> JP -> GA -> SR
Distancia estimada: 435.8 km


# Segunda parte

Resolver el problema agregando heurística para los siguientes destinos:

- San Rafael (SR)
- El Nihuil (EN)
- El sosneado (ES)
- Villa Atuel (VA)
- Jaime Prats (JP)
- Las Malvinas (LM)
- Salto de las Rosas (SDLR)
- Rama caída (RC)
- Monte Coman (MC)
- Gral. Alvear (GA)
- Rincón del Atuel (RA)

In [2]:
import json

with open('distances.json', 'r') as file:
    distances = json.loads(file.read())

all_cities = list(distances.keys())
all_cities

['SR', 'EN', 'ES', 'JP', 'VA', 'LM', 'SDLR', 'RC', 'MC', 'GA', 'RA']

In [3]:
def get_unvisited_cities() -> list:
    return [city for city in all_cities if city not in visited_cities]


def get_closest_neighbour(current_city: str) -> tuple:
    """
    Devuelve la ciudad vecina mas cercana que aún no se haya visitado.
    El formato de retorno es una tupla indicando (ciudad, distancia)
    """

    # Evaluamos solo las ciudades no visitadas
    possible_neighbours = get_unvisited_cities()
    # Empezamos considerando que la mejor opción, es la primera
    neighbour, distance = possible_neighbours[0], distances[current_city][possible_neighbours[0]]
    for candidate in possible_neighbours:
        # Si encontramos una opción mejor, usamos esa
        if distances[current_city][candidate] < distance:
            distance = distances[current_city][candidate]
            neighbour = candidate
    # Devolvemos la mejor opción encontrada
    return neighbour, distance


total_distance = 0.0
visited_cities = ['SR']
current_city = 'SR'

# Mientras queden ciudades por visitar
while len(all_cities) > len(visited_cities):
    # Avanzamos a la ciudad mas cercana no visitada
    next_destiny, distance = get_closest_neighbour(current_city)
    # Sumamos la distancia al total del recorrido
    total_distance += distance
    # Marcamos esa ciudad como la actual, para evaluar el próximo viaje
    current_city = next_destiny
    # Marcamos la ciudad como visitada
    visited_cities.append(current_city)

# Sumamos el viaje de vuelta a San Rafael, ya que nuestro while termina un paso antes
total_distance += distances[visited_cities[-1]]["SR"]
visited_cities.append("SR")

print(total_distance)
print(' -> '.join(visited_cities))

502.8
SR -> RC -> SDLR -> LM -> VA -> JP -> GA -> MC -> RA -> EN -> ES -> SR


Responder:

- ¿Qué heurística se utilizó? 

<hr>

La heurística utilizada fue la del vecino mas cercano. En cada ciudad, se evaluaba el viaje posible de menor esfuerzo y se procedía a realizarlo hasta terminar el recorrido. La solución rara vez será la óptima, pero el algoritmo es mucho mas eficiente y escalable que comprobar cada posible recorrido como el del primer ejercicio